# L5a: Maximum Flow Problems
In this lecture, we will explore maximum flow problems, which are a fundamental class of optimization problems in graph theory.

> __Learning Objectives__
>
> At the end of this lecture, you should be able to:
> * __Formulate__ maximum flow problems as flow networks and identify their applications in resource allocation and logistics. You'll use this to model real problems like transportation networks, supply chains, and worker assignments.
> * __Apply__ the Ford-Fulkerson method to solve maximum flow problems by finding augmenting paths. This algorithm helps you solve optimization problems in engineering, computer science, and operations research.
> * __Construct__ residual graphs and explain how they enable the Ford-Fulkerson algorithm to find optimal solutions. This helps you understand how flow algorithms work and debug network flows.
>
> We will examine the Ford-Fulkerson method, a classic approach for solving maximum flow problems.

This is a super cool topic, so let's dive in!
___

## Examples
Today, we will be using the following example(s) to illustrate key concepts:

> [▶ Understanding Maximum Flow Calculations in a Bipartite Graph using the Ford-Fulkerson Algorithm](CHEME-5800-L5a-FF-Example-MaxFlow-Fall-2025.ipynb). In this example, we will build a flow network from a workers→tasks edgelist, compute the maximum flow using the Ford–Fulkerson algorithm, and experiment with edge capacities to see how bottlenecks form and move.
___

## Motivation
Maximum flow problems enable us to find the largest amount of flow that can be transported from a source to a destination through a network with capacity constraints. These problems appear in diverse applications such as optimizing data transmission through computer networks, maximizing throughput in manufacturing pipelines, or determining optimal resource allocation in logistics systems.

Before diving into algorithms, let's establish what exactly constitutes a maximum flow problem.

> **Maximum Flow Problem**
>
> Consider a directed graph $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ where $\mathcal{V}$ represents the set of vertices (nodes) and $\mathcal{E}$ represents the set of directed edges. Each edge $(u,v) \in \mathcal{E}$ has an associated capacity $c(u,v) \geq 0$ that represents the maximum amount of flow that can pass through that edge.
>
> The network has a designated __source vertex__ $s \in \mathcal{V}$ where flow originates and a __sink vertex__ $t \in \mathcal{V}$ where flow terminates. We assume $s \neq t$ and that every vertex lies on some path from $s$ to $t$.
>
> A __flow__ is a function $f: \mathcal{E} \to \mathbb{R}$ that assigns a non-negative value to each edge, subject to two fundamental constraints:
>
> 1. **Capacity Constraint**: For all edges $(u,v) \in \mathcal{E}$: $0 \leq f(u,v) \leq c(u,v)$
> 2. **Flow Conservation**: For all vertices $v \in \mathcal{V} \setminus \{s,t\}$: $\sum_{u:(u,v) \in \mathcal{E}} f(u,v) = \sum_{w:(v,w) \in \mathcal{E}} f(v,w)$
>
> The __value__ of a flow $f$ is the total amount leaving the source: $|f| = \sum_{v:(s,v) \in \mathcal{E}} f(s,v) - \sum_{u:(u,s) \in \mathcal{E}} f(u,s)$
>
> The __maximum flow problem__ seeks to find a flow $f^*$ that maximizes this value:
> $$f^* = \arg\max_{f} |f|$$
>
> subject to the capacity and conservation constraints above.

> **Key Concepts:**
>
> * **Flow** represents any abstract quantity that moves through the network - data packets, water, materials, or even assignments of resources to tasks
> * **Capacity constraints** model physical or logical limits on how much can pass through each connection
> * **Flow conservation** ensures that flow is neither created nor destroyed at intermediate nodes - everything that flows in must flow out
> * **Source and sink** represent the origin and destination of the flow, where conservation doesn't apply since flow is generated and consumed respectively

This mathematical framework provides the foundation for understanding algorithms like Ford-Fulkerson that systematically find maximum flows by identifying augmenting paths through the network.
___

## Ford-Fulkerson Method
The Ford-Fulkerson _method_ is a greedy approach for computing the maximum flow through a network. It uses the concept of augmenting paths to iteratively increase the flow until no further augmenting paths can be identified.
* _Why method instead of algorithm?_ The Ford-Fulkerson method is a general approach to solving maximum flow problems, and it can be implemented using different algorithms to find augmenting paths. The choice of algorithm for finding these paths can affect the efficiency and correctness of the method.
* _What is an augmenting path?_ An augmenting path is a path from the source node $s\in\mathcal{V}$ to the sink node $t\in\mathcal{V}$ in a flow network where additional flow can be sent. The flow along this path can be increased, thus augmenting the total flow in the network.

First, we describe the Ford–Fulkerson method in general; then we specialize to the Edmonds–Karp variant, which uses breadth-first search (BFS) to find augmenting paths.

__Initialization__: Graph $\mathcal{G}=(\mathcal{V},\mathcal{E})$, source node $s\in\mathcal{V}$, sink node $t\in\mathcal{V}$, and edge capacities $\texttt{capacity}(u,v)\geq{0}$. Initialize a flow map $\texttt{flow}:\mathcal{E}\to\mathbb{R}$, where $\texttt{flow}(u,v)$ represents the flow from node $u$ to node $v$. The flow is initialized to zero for all edges: $\texttt{flow}(u,v)\gets{0}$ for all $(u,v)\in\mathcal{E}$.

While there exists a path $P$ from $s$ to $t$ in residual graph $\mathcal{G}_f$ (see "What is a Residual Graph?" below) __do__:
* Find $\Delta\gets\min\left\{ \texttt{capacity}(u,v)\, -\,\texttt{flow}(u,v) : (u,v) \in P \right\}$.
* For each edge $(u,v)\in P$ __do__:
    * Update the forward flow: $\texttt{flow}(u,v)\gets\texttt{flow}(u,v)+\Delta$
    * Update the reverse flow: $\texttt{flow}(v,u)\gets\texttt{flow}(v,u)-\Delta$

Although this description appears concise, there are several important concepts to understand. Let's begin by examining the most fundamental question: what is the residual graph $\mathcal{G}_f$?

### What is a Residual Graph?
The _residual graph_ $\mathcal G_f$ for a $\texttt{flow}(u,v)$ on the _original_ network $\mathcal G=(\mathcal{V},\mathcal{E})$ is defined as:
* **Vertex set**: The _residual graph_ $\mathcal G_f$ has the same vertex set as the original graph, i.e., the original set $\mathcal{V}$.
* **Residual edge set**: The residual edge set is defined as $\mathcal{E}_f \;=\;\bigl\{(u,v)\in \mathcal{E} : \texttt{residual}(u,v)>0\bigr\}$, where the _residual capacity_ $\texttt{residual}(u,v)$ is defined as:
$$
\texttt{residual}(u,v) \;=\;
\begin{cases}
    \texttt{capacity}(u,v) - \texttt{flow}(u,v), & \text{if }(u,v)\in \mathcal{E}\quad(\text{forward edge}),\\
    \texttt{flow}(v,u),          & \text{if }(v,u)\in \mathcal{E}\quad(\text{backward edge}),\\
    0,               & \text{otherwise.}
\end{cases}
$$

Any pair $(u,v)$ with $\texttt{residual}(u,v)>0$ appears in $\mathcal G_f$.

In $\mathcal G_f$, an **augmenting path** is any path from $s\to t$ using only edges in the edge set $\mathcal{E}_f$. Since the flow along this path can be increased, it augments the total flow in the network. But how do we find these augmenting paths? 

### How do we find augmenting paths?
Let's assume we constructed the residual graph $\mathcal{G}_f$ as described above. Then, we can find augmenting paths using a search algorithm, such as breadth-first search (BFS) or depth-first search (DFS)!
* __Edmonds-Karp algorithm__: The Edmonds-Karp algorithm is a specific implementation of the Ford-Fulkerson method that uses breadth-first search (BFS) to find augmenting paths. It guarantees that the maximum flow is found in $\mathcal{O}(|\mathcal{V}|\cdot|\mathcal{E}|^2)$ time, which is polynomial in the size of the graph.
* __What about Depth-First Search?__ Using depth-first search (DFS) to find augmenting paths is also possible; this is essentially the generic Ford-Fulkerson algorithm. However, it does not guarantee polynomial-time complexity and can lead to taking a very large (perhaps exponentially large) number of steps. Thus, we will not cover it here.

Ok, let's look at an example. 

> __Example__
> 
> [▶ Understanding Maximum Flow Calculations in a Bipartite Graph using the Ford-Fulkerson Algorithm](CHEME-5800-L5a-FF-Example-MaxFlow-Fall-2025.ipynb). In this example, we will build a flow network from a workers→tasks edgelist, compute the maximum flow using the Ford–Fulkerson algorithm, and experiment with edge capacities to see how bottlenecks form and move.

___

## Lab Exercises
In the lab L5b, we will implement use the Ford-Fulkerson method to find the maximum flow in a graph, exploring its properties and applications.

# Today?
That's all for today! What are three things you learned today? 
___